### This notebook I'll be showing how I got from raw data to the complete dataframe.
This does not replicate exactly into the dataframe showed in complete_dataframe.csv. In complete_dataframe I applied a technique to normalize the data,and created a target column which I called high_risk. So here you'll learn how to acess to transform the raw data into the dataset.

In case you missed it I strongly advise you to read the description and article of this database.

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install xlrd

In [ ]:
#Letting all names uniform so we can join all the informaiton later
df_patients = pd.read_excel('/kaggle/input/lung-cancer-patients-mrna-microarray/raw/DCLungStudy_Clinical_Covariates_with_Hgrade.xls')

df_origins = df_patients[['BlindedIDs', 'MICROARRAY']].loc[
    ~df_patients.BlindedIDs.isin(['None4TestSets'])].set_index('BlindedIDs').to_dict()[
    'MICROARRAY']

#Patients that are not in the study because of exam quality
patients_to_drop = pd.read_excel('/kaggle/input/lung-cancer-patients-mrna-microarray/raw/DCLungStudy_Not_In_Study_Reasons.xls')

Micro_array_to_drop = patients_to_drop['Chip name']
Micro_array_to_drop = Micro_array_to_drop.str.replace('MOFF_', 'Moff ')
Micro_array_to_drop = Micro_array_to_drop.str.replace('NCI_LUNG', 'NCI_Lung')
Micro_array_to_drop = Micro_array_to_drop.str.replace('NCI_lung', 'NCI_Lung')
df_patients.MICROARRAY = df_patients.MICROARRAY.str.replace('NCI_lung', 'NCI_Lung')
df_patients.MICROARRAY = df_patients.MICROARRAY.str.replace('NCI_Lung_', 'NCI_Lung')
df_patients.MICROARRAY = df_patients.MICROARRAY.str.replace('Control', 'control').str.replace(' ', '')
Micro_array_to_drop = Micro_array_to_drop.str.replace('NCI_Lung_', 'NCI_Lung')
Micro_array_to_drop = Micro_array_to_drop.str.replace('CONTROL', 'control').str.replace(' ', '')

df_patients = df_patients[(df_patients.MICROARRAY.isin(list(Micro_array_to_drop)) == False)]

In [ ]:
#Only patients with time of death or last contact!
df_patients = df_patients.replace('na', np.nan) #Another  for NaN
df_patients = df_patients[df_patients['MONTHS_TO_LAST_CONTACT_OR_DEATH'].notnull()]
df_patients['MONTHS_TO_LAST_CONTACT_OR_DEATH'] = df_patients['MONTHS_TO_LAST_CONTACT_OR_DEATH'].apply(
    lambda x: int(x))

#### By now we have imported patients data and replaced some annoying typos to turn this dataset more approachable. Now we we'll import gene data! 

In [ ]:
plt.style.use('ggplot')

In [ ]:
plt.figure(figsize=(12, 8))
plt.boxplot(df_patients['MONTHS_TO_LAST_CONTACT_OR_DEATH'])
plt.show()

In [ ]:
print('Life time mean: {} \nStandard Deviation: {}'.format(df_patients['MONTHS_TO_LAST_CONTACT_OR_DEATH'].mean(), df_patients['MONTHS_TO_LAST_CONTACT_OR_DEATH'].std()))

In [ ]:
mc_MSKCC = pd.read_excel('/kaggle/input/lung-cancer-patients-mrna-microarray/raw/DCLungStudy_dChip-Processed_microarray_data.xls', sheet_name = 'MSKCC', index_col = 'PROBE').T
mc_MSKCC

In [ ]:
mc_DFCI = pd.read_excel('/kaggle/input/lung-cancer-patients-mrna-microarray/raw/DCLungStudy_dChip-Processed_microarray_data.xls', sheet_name = 'CanadaDFCI', index_col = 'PROBE').T

mc_MICH = pd.read_excel('/kaggle/input/lung-cancer-patients-mrna-microarray/raw/DCLungStudy_dChip-Processed_microarray_data.xls', header=1, sheet_name = 'MICH', index_col = 'RANDOM_NUMBER_IDENTIFIER').T

mc_HLM = pd.read_excel('/kaggle/input/lung-cancer-patients-mrna-microarray/raw/DCLungStudy_dChip-Processed_microarray_data.xls', header=1, sheet_name = 'HLM', index_col = 'RANDOM_NUMBER_IDENTIFIER').T

In [ ]:
mc_MICH

In [ ]:
#Producing our dataset with only microarray expression
micro_array = pd.concat([mc_MSKCC, mc_HLM, mc_MICH, mc_DFCI])

#Renaming indexes
micro_array = micro_array.rename(df_origins)

#Adjusting indexes
micro_array.index = micro_array.index.str.replace(
    'NCI_LUNG', 'NCI_Lung').str.replace(
    'MOFF_', 'Moff ').str.replace(
    'NCI_lung', 'NCI_Lung').str.replace(
    'NCI_Lung_', 'NCI_Lung').str.replace(
    'Control', 'control').str.replace(
    'CONTROL', 'control').str.replace(
    'REP', 'Rep').str.replace(' ', '')

#Now droping the guys we previously dropped
micro_array = micro_array.loc[micro_array.index.isin(list(df_patients.MICROARRAY))]

#Creating targets
micro_array = micro_array.dropna(axis=1).dropna()
micro_array.shape

In [ ]:
df = df_patients.set_index('MICROARRAY').join(micro_array)

## It's Done!
This notebook does not have much data "sciencing" or data "analysing", since this is an annoying dataframe to deal with, I uploaded the steps I used to get to a "raw" dataframe. 
* Ps: The column MONTHS_TO_LAST_CONTACT_OR_DEATH was acquired after the mRNA exam, so its usefull to create a model to predict survivability. I defined less than 36 months initialy because it was the life expectancy of a lung cancer diagnosed patient, but feel free to manipulate this target and let me know what you can come up with :)